<a href="https://colab.research.google.com/github/vlochub/MIT-Xpro-colab/blob/main/retail_pricing_model_update_vevek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Analytics Edge of Retail Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [ ]:
!pip install gurobipy

# load require packages

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [ ]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,164))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163]
157
158


### Parameters

In [ ]:
#From orginal price / sales(demand) data
# Developed linear demand model: used 27-156 weeks (3 years) of data; 27-104( years 1/2)Train, 105-156( last year)Test

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model
# model: 𝑑_𝑡 = 1.978242858 -2.809634145 *𝑝_𝑡 + 0.963410728*𝑝_(𝑡−1) + 0.759639170*𝑝_(𝑡−2) + -0.562046910*𝑆𝑒𝑎𝑠𝑜𝑛2 + 0.087545274*𝑆𝑒𝑎𝑠𝑜𝑛3 + -0.402637480*𝑆𝑒𝑎𝑠𝑜𝑛4 + -0.027326010*𝑆𝑒𝑎𝑠𝑜𝑛5 + 0.004349885*𝑆𝑒𝑎𝑠𝑜𝑛6 + -0.036102297*𝑆𝑒𝑎𝑠𝑜𝑛7 + -0.069280527*𝑆𝑒𝑎𝑠𝑜𝑛8 + 0.160276197*𝑆𝑒𝑎𝑠𝑜𝑛9 + 1.104208897*𝑆𝑒𝑎𝑠𝑜𝑛10 + 1.122711287*𝑆𝑒𝑎𝑠𝑜𝑛11 + 1.176802194*𝑆𝑒𝑎𝑠𝑜𝑛12 + 0.947945548*𝑆𝑒𝑎𝑠𝑜𝑛13
# Store model parameters
intercept = 1.978242858
# causal factors
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
# time series factor
season_coeff = {1: 0, 2: -0.562046910, 3: 0.087545274,  4: -0.402637480, 5: -0.027326010, 6: 0.004349885,
               7: -0.036102297, 8: -0.069280527, 9: 0.160276197, 10: 1.104208897, 11: 1.122711287, 12: 1.176802194, 13: 0.947945548}

In [ ]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: np.float64(1.0), 158: np.float64(1.0), 159: np.float64(1.0), 160: np.float64(1.0), 161: np.float64(2.0), 162: np.float64(2.0), 163: np.float64(2.0)}


## Model 1 - no price ladder or business rules
#### DECISION VARIABLES:  prices in weeks 157 – 169  and demand in weeks 157 – 169
######  Prices; 𝑝_𝑡  for  t = 157, … , 169   and   𝑑_𝑡  for  t = 157, … , 169
######  Demand; follows linearly additive model:
#### MAXIMIZE 𝑅𝑒𝑣𝑒𝑛𝑢𝑒= 𝑝157𝑑157 + 𝑝158𝑑158 + 𝑝159𝑑159 + ... + 𝑝169𝑑169
#### Objective function,  sales revenue in next quarter (weeks 157 – 169)

1.   List item
2.   List item


#### SUBJECT TO CONSTRAINTS:
#### Demand is characterized by our linearly additive model:
#### 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13


In [ ]:
# Create Gurobi model object - repository for all objects to be used in the model
mod1 = gp.Model ("price_model_1")

In [ ]:
# Define decision variables
p = mod1.addVars(weeks, ub = 1)

In [ ]:
p

{157: <gurobi.Var *Awaiting Model Update*>,
 158: <gurobi.Var *Awaiting Model Update*>,
 159: <gurobi.Var *Awaiting Model Update*>,
 160: <gurobi.Var *Awaiting Model Update*>,
 161: <gurobi.Var *Awaiting Model Update*>,
 162: <gurobi.Var *Awaiting Model Update*>,
 163: <gurobi.Var *Awaiting Model Update*>}

In [ ]:
# Set objective function
# Example first 2 lines fully written out,
# Short cut for writing out each line
#  after the first 2 weeks, the remaining weeks[2:], are developed using the sum() and a for loop  (weeks 159 on...)
#    price at week(wx) * demand (from linear demand fucntion at weekx)
obj_fn = mod1.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
mod1.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 7 columns and 0 nonzeros
Model fingerprint: 0x6c343c6b
Model has 18 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective 5.77437929e+00


In [ ]:
p[157] # to see 1st prediction

<gurobi.Var C0 (value 0.9299494936400734)>

In [ ]:
p[158] # to see 2nd prediction

<gurobi.Var C1 (value 0.9036165540216698)>

In [ ]:
# to see all predictions and save to a dataframe
df1 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df1.loc[w,"price"] = p[w].x
df1

,price
157,0.929949
158,0.903617
159,0.860658
160,0.809188
161,0.660137
162,0.549248
163,0.435433


In [ ]:
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 2 - no business rules + price ladder

In [ ]:
p_ladder = [1.0, 0.95, 0.85, 0.75, 0.60, 0.50] #create a list for price ladder

In [ ]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [ ]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [ ]:
mod2.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 14 rows, 49 columns and 91 nonzeros
Model fingerprint: 0x93a74482
Model has 18 quadratic objective terms
Variable types: 7 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 14 rows, 42 columns, 77 nonzeros
Presolved model has 18 quadratic objective terms

Continuing optimization...


Explored 28 nodes (113 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 10: 5.73818 5.73214 5.72735 ... 5.61884

Optimal solution found (tolerance 1.00e-04)
Best objective 5.738181277773e+00, best bou

In [ ]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,0.95
158,0.95
159,0.85
160,0.85
161,0.75
162,0.6
163,0.5


In [ ]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()